# Оценка стоимости опционов с помощью Julia 

В данной работе мы рассмотрим инструменты, которые реализованы в Julia, для стоимостной оценки различных видов опционов.

**Выполнили:**

Пономаренко Анна

Никитанова Мария

Лобанов Александр

In [1]:
#Используемые пакеты
using Pkg;
using Distributions;
using Random;
using Dates; 

Pkg.add(PackageSpec(url="https://gitlab.com/rcalxrc08/FinancialMonteCarlo.jl"))
using FinancialMonteCarlo;

Pkg.add("QuantLib")
using QuantLib;

   Updating git-repo `https://gitlab.com/rcalxrc08/FinancialMonteCarlo.jl`
   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


Опцион - популярный инструмент хеджирования рисков. Стоимость опциона напрямую зависит от его вида. В зависимости, какую дату оферты предложит продавец опциона, по какому страйку и какого типа: put или call, цена может варьироваться в крупном диапазоне. 

В целом, способы оценки опционов можно разделить на 2 группы: использующие аналитические или численные методы компьютации. Мы рассмотрим обе эти группы, и что для них предлагает Julia) 

In [15]:
# Библиотека содержит в себе большое количество встроенных функций, которые позволяют оценивать Американские, Европейские и Бермудские опционы
#Для оценки необходимо использовать несколько обязательных пунктов, которые также содержатся внутри пакета: 
#Процентные ставки (структура и безрисковая в текущей экономике), дата исполнения опциона, дата рассмотрения его цены (в какой момент подвергается оценке), цены – страйковая и базового актива и др.)
#Библиотека подстроена под введение данных параметров

#По умолчанию можно использовать модель Блэка-Шоулза, однако существуют и более сложные методы оценки, которым необходимо большее количество входных параметров, поэтому зададим их все в самом начале

#Для ответа необходимо лишь выбрать нужный способ оценки и, возможно, изменить часть параметров

## Аналитические методы оценки опционов

In [13]:
function main() 
  cal = QuantLib.Time.TargetCalendar() # Поскольку цена опциона зависит от той даты, на которую мы смотрим. И также от той даты, когда он был выпущен. Дата в формате год, месяц, день
  todaysDate = Date(2021, 1, 30)
  settlementDate = Date(1998, 5, 17) # Дата размещения
  
  set_eval_date!(settings, todaysDate)
  
  optionType = Put() # Выбираем тип опциона
  underlying = 36.0 # Цена актива, находящегося в нем
  strike = 40.0 # Страйковая цена (по которой реализуется)
  dividendYield = 0.00 # Дивиденды по нему. Для простоты можем взять равные нулю
  riskFreeRate = 0.06 # Безрисковая ставка в экономике
  vol = 0.20 # Волатильность 
  mat = Date(1999, 5, 17) # Maturity date 
  dc = QuantLib.Time.Actual365()

  exerciseDates = Date[settlementDate + Dates.Month(3 * i) for i = 1:4]

  europeanEx = EuropeanEx(mat)
  bermudanEx = BermudanEx(exerciseDates)
  americanEx = AmericanEx(settlementDate, mat)

  underlyingH = Quote(underlying)

  flatTermStructure = FlatForwardTermStructure(settlementDate, cal, Quote(riskFreeRate), dc) # Структура процентных ставок
  flatDividendTS = FlatForwardTermStructure(settlementDate, cal, Quote(dividendYield), dc) # Структура дивидендов
  flatVolTS = BlackConstantVol(settlementDate, cal, vol, dc) # Структура волатильности 

  payoff = PlainVanillaPayoff(optionType, strike) # Выплаты по ванильному опциону

  bsmProcess = BlackScholesMertonProcess(underlyingH, flatTermStructure, flatDividendTS, flatVolTS) # Блэк-Шоулз процесс

  # options
  bsPE = AnalyticEuropeanEngine(bsmProcess) # black scholes PE
  europeanOption = VanillaOption(payoff, europeanEx, bsPE)

  # black scholes for European
  println("BlackScholes (Euro):                   ", npv(europeanOption))

  # heston process #Модель Хестона
  hestonProcess = HestonProcess(flatTermStructure, flatDividendTS, underlyingH, vol * vol, 1.0, vol * vol, 0.001, 0.0)
  hestonModel = HestonModel(hestonProcess)
  hestonPE = AnalyticHestonEngine(hestonModel)

  europeanOption = update_pricing_engine(europeanOption, hestonPE)
  println("Heston semi-analytic (Euro):           ", npv(europeanOption))

  # bates process #SVJ модель
  batesProcess = BatesProcess(flatTermStructure, flatDividendTS, underlyingH, vol * vol, 1.0, vol * vol,
                              0.001, 0.0, 1e-14, 1e-14, 1e-14)
  batesModel = BatesModel(batesProcess)
  batesPE = BatesEngine(batesModel)

  europeanOption = update_pricing_engine(europeanOption, batesPE)
  println("Bates semi-analytic (Euro):            ", npv(europeanOption))

  # American Option # Для американских опционов модели
  baroneAdesiPE = BaroneAdesiWhaleyApproximationEngine(bsmProcess)
  americanOption = VanillaOption(payoff, americanEx, baroneAdesiPE)
  println("Barone-Adesi/Whaley (Amer):            ", npv(americanOption))

  # Bjerksund Stensland engine # Модель 1993 года от авторов Bjerksund и Stensland
  bjerksundStenslandPE = BjerksundStenslandApproximationEngine(bsmProcess)
  americanOption = update_pricing_engine(americanOption, bjerksundStenslandPE)
  println("Bjerksund Stensland (Amer):            ", npv(americanOption))

  # Bermudan Option для Бермудского опциона
  fdBermudanPE = FDBermudanEngine(bsmProcess, CrankNelson, timeSteps, timeSteps - 1)
  bermudanOption = VanillaOption(payoff, bermudanExercise, fdBermudanPE)
  println("FD (Berm):                             ", npv(bermudanOption))

  fdAmericanPE = FDAmericanEngine(bsmProcess, CrankNelson, timeSteps, timeSteps - 1)
  americanOption = update_pricing_engine(americanOption, fdAmericanPE)
  println("FD (Amer):                             ", npv(americanOption))

  # Tian # Биномиальная модель Тиан
  btTianEuro = BinomialVanillaEngine(bsmProcess, timeSteps, Tian)
  europeanOption = update_pricing_engine(europeanOption, btTianEuro)
  println("Binomial Tian (Euro):                  ", npv(europeanOption))

  btTianBermudan = BinomialVanillaEngine(bsmProcess, timeSteps, Tian)
  bermudanOption = update_pricing_engine(bermudanOption, btTianBermudan)
  println("Binomial Tian (Berm):                  ", npv(bermudanOption))

  btTianAmerican = BinomialVanillaEngine(bsmProcess, timeSteps, Tian)
  americanOption = update_pricing_engine(americanOption, btTianAmerican)
  println("Binomial Tian (Amer):                  ", npv(americanOption))

  # LeisenReimer (суть модели – базовое биномиальное дерево цены сосредоточено вокруг цены исполнения опциона на момент истечения срока)
  btLeisenReimerEuro = BinomialVanillaEngine(bsmProcess, timeSteps, LeisenReimer)
  europeanOption = update_pricing_engine(europeanOption, btLeisenReimerEuro)
  println("Binomial Leisen-Reimer (Euro):         ", npv(europeanOption))

  btLeisenReimerBermudan = BinomialVanillaEngine(bsmProcess, timeSteps, LeisenReimer)
  bermudanOption = update_pricing_engine(bermudanOption, btLeisenReimerBermudan)
  println("Binomial Leisen-Reimer (Berm):         ", npv(bermudanOption))

  btLeisenReimerAmerican = BinomialVanillaEngine(bsmProcess, timeSteps, LeisenReimer)
  americanOption = update_pricing_engine(americanOption, btLeisenReimerAmerican)
  println("Binomial Leisen-Reimer (Amer):         ", npv(americanOption))

  ## MONTE CARLO ##
  timeSteps = 1
  mcSeed = 42
  mcengine1 = MCEuropeanEngine(bsmProcess; timeSteps = timeSteps, requiredTolerance = 0.02, seed = mcSeed)
  europeanOption = update_pricing_engine(europeanOption, mcengine1)
  println("MC-crude (Euro):                       ", npv(europeanOption))

  nSamples = 32768
  mcengine2 = MCEuropeanEngine(bsmProcess; timeSteps = timeSteps, requiredSamples = nSamples)
  europeanOption = update_pricing_engine(europeanOption, mcengine2)
  println("MC-Sobol (Euro):                       ", npv(europeanOption))

  mcengine3 = MCAmericanEngine(bsmProcess; timeSteps = 100, antitheticVariate=true, requiredTolerance=0.02, seed=mcSeed, nCalibrationSamples=4096)
  americanOption = update_pricing_engine(americanOption, mcengine3)
  println("MC-Longstaff Schwartz (Amer):          ", npv(americanOption))
  # americanOption
end

main (generic function with 1 method)

## Численные методы оценки опционов

Широкое распространение на российском рынке получили ванильные опционы. Они просто оцениваются с помощью аналитических методов и не требуют больших вычислительных мощностей для этого. Экзотические опционы выделяются особенностями условий на которых они торгуются (в России они составляют 5% от объема торгов опционами). Для таких финансовых инструментов сложно вывести их стоимость аналитически, зато она отлично аппроксимируется с помощью численных методов. Одним из таких наиболее распространенных методов является метод Монте-Карло, благодаря которому симулируются все возможные движения цены инструмента и вычисляется ее математическое ожидание. 

Далее будут рассмотрены инструменты, которые реализованы в Julia для реализации оценки различных видов опционов с помощью симуляций Монте Карло.

In [3]:
# Устанавливаем на компьютер необходимые пакеты 
using Pkg 

#Pkg.add("DataFrames")

#Pkg.add("CSV")

#Pkg.add("Econometrics")

#Pkg.add(PackageSpec(url="https://gitlab.com/rcalxrc08/FinancialMonteCarlo.jl


#Pkg.add("Distributions")

## Симулирование чисел из различных распределений

Нормальное распределение

In [4]:
d = Normal(50, 7)
x = rand(d, 10)

10-element Array{Float64,1}:
 51.37206411612233
 62.817130580885944
 44.10741067249985
 53.90977386227923
 39.67847240820517
 56.17901894218736
 44.53891707671961
 39.1296509129932
 48.91690818949762
 50.03479503600993

Распределение Бернулли

In [5]:
d = Bernoulli(4)
x = rand(d, 100)

LoadError: ArgumentError: Bernoulli: the condition zero(p) <= p <= one(p) is not satisfied.

### Метод Монте-Карло для оценивания стоимости опциона

num_sims - количество симуляций

S - текущая цена

T - количество прошедших периодов

r - ожидаемая доходность

K - страйк

v - Волатильность 

types  -  тип опциона 

In [9]:
function monte_carlo_price(types, num_sims, S, K, r, v, T)     
    
    #скорректируем цену на волатильность 
    S_adj = S * exp(T*(r-0.5*v*v))
    #cчетчик текущей цены
    S_cur = 0.0
    #выигрыш 
    payoff = 0.0

    #итерации Монте-Карло
    for i = 1:num_sims
        #генирируем число из стандартного нормального распределения 
        gauss_bm = randn()
        #Высчитаем новую цену на период 
        S_cur = S_adj * exp(sqrt(v*v*T) * gauss_bm)
        payoff += ifelse(types == "Call", ifelse(S_cur-K > 0.0, S_cur-K, 0.0), ifelse(-S_cur+K > 0.0, -S_cur+K, 0.0))
    end

    res = payoff/num_sims * exp(-r*T) 
end

monte_carlo_price (generic function with 1 method)

In [12]:
types = "Call"
num_sims = 1000000000
cur_price = 100.0
strike = 100.0
r = 0.05 
std = 0.2 #cтандартное отклонение доходности по предыдущим данным 
T = 1 #период до оферты

y = monte_carlo_price(types, num_sims, cur_price, strike, r, std, T)
println(y)

10.450633318730954


In [16]:
# По итогам работы в конце будет добавлен метод оценки опционов с помощью модели Блэка-Шоулза и построены графики, демонстрирующие выводы